In [ ]:
!pip install datasets
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import re
import pandas as pd
import concurrent.futures

dataset = load_dataset(path="joonhok-exo-ai/korean_law_open_data_precedents", split="train")
dataset_df = pd.DataFrame(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/85660 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

# 해당 키워드가 들어간 사건명을 검색하고 key값 저장
keywords = {
    "살인":{"살인"},
    "절도":{"절도","불법사용","침입절도"},
    "장물":{"장물"},
    "사기":{"사기","컴퓨터등사용사기","부당이득","편의시설부정이용","전기통신금융사기","보험사기"},
    "횡령":{"횡령"},
    "배임":{"배임"},
    "도로교통법":{"운전","도로"},
    "손괴":{"손괴"},
    "강도":{"강도"},
    "방화":{"방화"},
    "성폭력":{"성폭력","강간","특수강도강간","카메라등이용촬영","공중밀집에서의추행","음란물"},
    "폭행":{"폭행","폭력행위"},
    "상해":{"상해"},
    "협박":{"협박"},
    "공갈":{"공갈"},
    "약취/유인":{"약취","유인","부녀매매","국외이송"},
    "체포/감금":{"체포","감금"},
    "위조범죄":{"통화","유가","인지","우표","문서","인장"},
    "공무원범죄":{"직무","직권","수뢰","증뢰"},
    "도박":{"도박","복표"},
    "과실범죄":{"과실","실화"},
    "명예":{"명예"},
    "권리행사방해":{"권리행사방해"},
    "주거침입":{"주거침입"},
    "비밀침해":{"비밀침해"},
    "유기":{"유기"},
    "교통방해":{"교통방해"},
    "도주/범죄은닉":{"도주","은닉"},
    "위증과증거인멸":{"위증","증거인멸"},
    "무고":{"무고"},
    "가정폭력":{"가정폭력"},
    "개인정보보호법":{"개인정보"},
    "건설":{"건설","건축"},
    "게임산업진흥법":{"게임"},
    "고용보험법":{"고용보험"},
    "공유수면관리":{"공유수면"},
    "공인중개사법":{"공인중개"},
    "공중위생법":{"공중위생"},
    "공직선거법":{"공직선거"},
    "관세법":{"관세"},
    "교통사고처리특례법":{"교통사고"},
    "국가기술자격법":{"국가기술자격"},
    "국가보안법":{"국가보안"},
    "국민체육진흥법":{"국민체육"},
    "근로기준법":{"근로","퇴직","급여"},
    "노동조합":{"노동"},
    "농수산물":{"농수산물"},
    "도시및주거환경":{"도시","주거환경"},
    "공정거래법":{"독점","공정거래"},
    "마약류관리에관한법률":{"마약"},
    "변호사법":{"변호사"},
    "병역법":{"병역"},
    "부동산":{"부동산"},
    "부정경쟁방지및영업비밀보호":{"부정경쟁","영업비밀"},
    "부정수표":{"부정수표"},
    "사해행위":{"사해행위"},
    "산업안전보건법":{"산업안전"},
    "산지관리법":{"산지"},
    "상표법":{"상표"},
    "선박":{"선박"},
    "성매매알선":{"성매매"},
    "식품위생법":{"식품위생"},
    "아동˙청소년":{"아동·청소년의성보호","아동"},
    "약사법":{"약사법"},
    "외국환거래법":{"외국환"},
    "의료법":{"의료"},
    "자동차":{"자동차"},
    "자본시장과금융투자업":{"자본시장","금융투자"},
    "저작권법":{"저작권"},
    "전자금융거래법":{"전자금융"},
    "정보통신망":{"정보통신망"},
    "조세범처벌법":{"조세범"},
    "주민등록법":{"주민등록법"},
    "주택법":{"주택"},
    "집회및시위":{"집회","시위"},
    "총포·도검·화약류단속":{"총포","도검","화약류"},
    "특허법":{"특허"},
    "폐기물관리법":{"폐기물"}
}
# 키워드를 범죄로 매핑
crime_to_keyword = {crime: keyword for keyword, crimes in keywords.items() for crime in crimes}

# 필터링된 사건 수집
filtered_cases = {}
for case in dataset_df['사건명'].unique():
    if case is not None:
        matched = False
        for crime, keyword in crime_to_keyword.items():
            if crime in case:
                if keyword in filtered_cases:
                    filtered_cases[keyword].add(case)
                else:
                    filtered_cases[keyword] = {case}
                matched = True
                break
        if not matched:
            if "기타" in filtered_cases:
                filtered_cases["기타"].add(case)
            else:
                filtered_cases["기타"] = {case}

# 모든 필터링된 사건을 중복 없이 통합
all_filtered_cases = set()
for cases in filtered_cases.values():
    all_filtered_cases.update(cases)

filtered_dataset_df = dataset_df[dataset_df['사건명'].isin(all_filtered_cases)]

# '분류' 열을 추가하고 키워드에 해당하는 값 할당
filtered_dataset_df['분류'] = filtered_dataset_df['사건명'].apply(lambda x: next((key for key, value in keywords.items() if any(keyword in x for keyword in value)), None))

# '기타'로 분류되는 행 추가
filtered_dataset_df.loc[filtered_dataset_df['분류'].isnull(), '분류'] = '기타'

filtered_dataset_df = filtered_dataset_df[filtered_dataset_df['분류'] != '기타']

#모든 사건명의 공백 제거
filtered_dataset_df['사건명'] = filtered_dataset_df['사건명'].str.replace(' ', '')

# 결과 출력
filtered_dataset_df

<ipython-input-22-736f20cd5ceb>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset_df['분류'] = filtered_dataset_df['사건명'].apply(lambda x: next((key for key, value in keywords.items() if any(keyword in x for keyword in value)), None))
<ipython-input-22-736f20cd5ceb>:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset_df['사건명'] = filtered_dataset_df['사건명'].str.replace(' ', '')


,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,참조조문,참조판례,전문,분류
2,85835,부동산소유권이전등기,4281민상362,19480407,선고,대법원,민사,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증 나. 계약해제존재의 가항변에 대한 판...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,"민사소송법 제80조 , 제137조, 제395조",None,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",부동산
7,85833,부동산이전등기말소,4281민상314,19490402,선고,대법원,민사,판결,부동산소유권이전 가등기권리자의 제3자에 대한 소유권이전 본등기의 말소청구권,가. 부동산소유권이전 가등기후에 등기의무자가 제3자에게 그 부동산소유권이전의 본등기...,부동산등기법 제2조,None,"【원고, 상고인】\n박희승\n【피고, 피상고인】\n조태환 외 32인\n【원심판결】\...",부동산
10,85839,국가보안법위반,4283형상72,19500320,선고,대법원,형사,판결,가. 연속범중의 일부행위에 대한 위법과 상고이유의 적부 나. (가) 남로당산하단체인...,가. 연속범을 구성하는 수개행위 중 그 일부행위에 대한 판단에 위법이었다 할 지라도...,"국가보안법 제4조, 제1조 제3호 , 형사소송법 제411조, 형법 제21조",None,"【상고인, 피고인】\n【원 심】\n서울고등법원\n【주 문】\n원판결을 파훼한다.\n...",국가보안법
11,215267,국가보안법제3조급법령제19호제4조나항각위반피고,4283형상7,19501120,선고,대법원,형사,판결,긴급구속사건과 사법경찰관의 피의자신문조서 작성권의 존부,피의자를 긴급구속한 사법경찰관은 형사소송법의 개정 제8조에 의하여 그 구속을 유지하...,"형사소송법 제8조, 제56조, 제343조",None,"【피고인, 상고인】\n피고인\n【변 호 인】\n김성재\n【주 문】\n본건 상고를 기...",국가보안법
12,85837,국가보안법위반,4283형상14,19501127,선고,대법원,형사,판결,국가보안법 제1조 제3호의 죄와 동법 제3조의 죄와의 죄질의 동일 여부,국가보안법 제1조 제3호의 결사는 국헌을 위배하여 정부를 잠칭하거나 국가를 변난할 ...,"국가보안법 제1조 제2항, 제3조 제2항 , 형법 제55조, 제45조",None,"【상고인, 피고인】\n【원심판결】\n제1심 부산지방법원, 제2심 동지방법원\n【주 ...",국가보안법
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85644,212537,종합부동산세등부과처분취소,2012두26852,20141127,선고,대법원,세무,판결,"[1] 신탁재산이 신탁법에 따른 신탁등기나 등록이 마쳐지지 아니한 경우, 신탁재산에...",[1] 구 지방세법(2010. 3. 31. 법률 제10221호로 전부 개정되기 전의...,[1] 구 지방세법(2010. 3. 31. 법률 제10221호로 전부 개정되기 전의...,"[1] 대법원 1993. 4. 27. 선고 92누8163 판결(공1993하, 160...","【원고, 피상고인】 상도134지역주택조합 (소송대리인 법무법인(유한) 한결 담당변호...",부동산
85645,179916,공직선거법위반·국가정보원법위반,2014노2820,20150209,선고,서울고등법원,형사,판결,None,None,None,None,"【피 고 인】\n【항 소 인】 피고인들, 검사\n【검 사】 진재선(기소), 박...",공직선거법
85646,188351,뇌물수수·뇌물공여·특정범죄가중처벌등에관한법률위반(뇌물)·변호사법위반·특정경제범죄가중...,"2014고합587, 441(병합), 603(병합), 802(병합)",20151127,선고,부산지방법원,형사,판결,None,None,None,None,"【피 고 인】\n【검 사】 전현민, 윤철민(기소), 천헌주, 서동민(공판)\n...",사기
85647,188019,사기·보조금관리에관한법률위반(일부인정된죄명보조금의예산및관리에관한법률위반),2015노4075,20160526,선고,부산지방법원,형사,판결,None,None,None,None,"【피 고 인】\n【항 소 인】 피고인들\n【검 사】 진호식, 권오승(각 기소)...",사기


In [ ]:
# '전문' 열에서 문장 추출 함수
def extract_sentence_between_markers(text):
    # "【주 문】"과 "【이 유】" 사이의 문자열 추출
    result = re.search(r'【주 문】(.*?)【이 유】', text, re.DOTALL)
    if result:
        # 추출된 문자열에서 "【"의 등장 횟수 계산
        if result.group(1).count("【") > 1:
            return None
        else:
            return result.group(1).strip()
    else:
        return None

In [ ]:
# '전문' 열의 데이터 타입 확인
print(filtered_dataset_df['전문'].dtype)

# '전문' 열의 모든 값을 문자열로 변환
filtered_dataset_df['전문'] = filtered_dataset_df['전문'].astype(str)

# 변환 후 '결과' 열 추가 (앞뒤 공백 제거)
filtered_dataset_df['판결'] = filtered_dataset_df['전문'].apply(lambda x: extract_sentence_between_markers(x.strip()))

# 결과 출력
filtered_dataset_df

# '판결' 열에서 NaN 값을 가진 행 제거
filtered_dataset_df = filtered_dataset_df.dropna(subset=['판결'])
# 복수의 피고인 존재하는 행 제거
mask = filtered_dataset_df['판결'].str.contains('피고인2|피고인 2', case=False, regex=True)
filtered_dataset_df = filtered_dataset_df[~mask]

object


In [ ]:
#징역 추출
def extract_sentence_length(sentence):
    """주어진 문장에서 '징역' 형량을 개월 단위로 추출하여 변환합니다. 소수점은 버립니다."""
    # '무기징역' 처리
    if '무기징역' in sentence:
        return '무기징역'

    # 단기/장기 형량 처리
    match = re.search(r'징역\s*장기\s*(\d+)\s*년', sentence)
    if match:
        long_term = int(match.group(1)) * 12  # 장기 형량을 개월로 변환
        return long_term*12

    # 일반적인 형량 처리
    match = re.search(r'징역\s*(\d+)\s*년\s*(\d+)?\s*(월|개월)?', sentence)
    if match:
        years = int(match.group(1)) if match.group(1) else 0
        months = int(match.group(2)) if match.group(2) else 0
        return years * 12 + months

    match = re.search(r'징역\s*(\d+)\s*(월|개월)', sentence)
    if match:
        return int(match.group(1))

    match = re.search(r'징역\s*(\d+)\s*년', sentence)
    if match:
        return int(match.group(1)) * 12

    return None

In [ ]:
filtered_dataset_df['징역(개월)'] = filtered_dataset_df['판결'].apply(extract_sentence_length)
filtered_dataset_df['징역(개월)'] = filtered_dataset_df['징역(개월)'].apply(lambda x: int(x) if isinstance(x, int) else x)
filtered_dataset_df[filtered_dataset_df['판결'].str.contains('징역')]

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,참조조문,참조판례,전문,분류,판결,징역(개월)
34,85843,독직상해,4285형비상1,19520422,선고,대법원,형사,판결,독직상해와 죄수,독직상해죄는 단순일죄이요 다만 그 처단에 있어서 형법 제10조에 의하여 이와 상해죄...,"형법 제196조, 제194조, 제204조, 제45조 , 형사소송법 제56조, 제52...",None,"【비상, 상고인】\n검찰총장\n【원 심】\n청주지방법원\n【주 문】\n원판결을 파훼...",상해,원판결을 파훼한다.\n피고인을 징역 8월에 처하고 1년간 우 형의 집행을 유예한다.,8
324,86117,상해치사,4288형상175,19550805,선고,대법원,형사,판결,구금일수와 필요적 산입,피고인의 공소를 파기함에 있어서 원심판결 선고전 구금일 수가 있으면 형법 제57조에...,"형법 제57조, 형사소송법 제482조",None,"【상고인, 피고인】\n【원심판결】\n제1심 부산지방법원 진주지원, 제2심 대구고등법...",상해,원판결을 파기한다\n피고인을 징역 2년에 처한다\n판결선고전 구금일수중 제1심 30...,24
396,86203,"공무상비밀표시무효,주거침입",4289형상5,19560305,선고,대법원,형사,판결,피고인의 항소와 불이익변경의 금지,제1심 판결이 2척의 공소범죄사실중 1척사실만을 인정하여 유죄를 선고하고 기여사실에...,"형사소송법 제368조, 제342조",None,"【상고인, 피고인】\n【변 호 인】\n변호사 이재원\n【원심판결】\n제1심 서울지방...",주거침입,원판결을 파기한다\n피고인을 징역4월에 처한다\n제1심 판결 선고전 구금일수중 80...,4
427,86206,"특수강도상해,특수강도미수등",4289형상118,19560619,선고,대법원,형사,판결,형법 제57조와 형사소송법 제482조와의 관계,형법 제57조에 의한 구금일수 통산은 형사소송법 제482조에 의하여 법정 통산을 할...,"형법 제57조, 형사소송법 제482조",None,"【상고인, 검사】\n【변 호 인】\n변호사 한격만\n【원심판결】\n제1심 부산지방법...",강도,원판결을 파기한다\n피고인을 징역 7년에 처한다\n제1심 판결 선고 전의 구금일수중...,84
438,86210,강도상해,4289형상170,19560817,선고,대법원,형사,판결,가. 절도죄와 재물의 악지 나. 타인의 혼수상태하에서 우발적으로 행한 도취죄의 태용,가. 설사 피해자가 졸도하여 의식을 상실한 경우에도 현장에 일실된 피해자의 물건은 ...,"형법 제329조, 제333조",None,"【상고인, 검사】\n김희주\n【원심판결】\n제1심 광주지방법원 목포지원, 제2심 광...",강도,원판결을 파기한다\n피고인을 징역 1년에 처한다\n제1심 판결 선고 전 구금일수 중...,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85303,233291,아동·청소년의성보호에관한법률위반(강간등치상)[인정된죄명아동·청소년의성보호에관한법률위...,2022노1599,20221108,선고,서울고등법원,형사,판결,None,None,None,None,"【피 고 인】\n피고인\n【항 소 인】\n쌍방\n【군 검 사】\n민경찬(검사, 기소...",성폭력,"원심판결을 파기한다.\n피고인을 징역 3년에 처한다.\n다만, 이 판결 확정일로부터...",36
85340,234269,특정범죄가중처벌등에관한법률위반(보복협박등)·스토킹범죄의처벌등에관한법률위반·가정폭력범...,2022노2157,20221124,선고,서울고법,형사,판결 : 확정,피고인이 별거 중인 배우자 甲을 찾아가 협박하였다는 가정폭력행위로 법원으로부터 10...,피고인이 별거 중인 배우자 甲을 찾아가 협박하였다는 가정폭력행위로 법원으로부터 10...,"스토킹범죄의 처벌 등에 관한 법률 제1조, 제2조 제1호 (다)목, 제2호, 제18...",None,【피 고 인】\n피고인\n【항 소 인】\n피고인\n【검 사】\n고현욱 외 1인\n【...,협박,원심판결을 파기한다.\n피고인을 징역 1년에 처한다.\n피고인에게 40시간의 스토킹...,12
85394,233811,마약류관리에관한법률위반(향정)·마약류관리에관한법률위반(대마),2022노331,20221222,선고,대구고법,형사,판결 : 확정,피고인이 ‘자신의 주거에서 향정신성의약품인 LSD를 투약하여 사용하였다.’는 마약류...,피고인이 ‘자신의 주거에서 향정신성의약품인 LSD 0.5장을 투약하여 사용하였다.’...,"마약류 관리에 관한 법률 제2조 제3호 (가)목, 제3조 제5호, 제59조 제1항 ...",None,【피 고 인】\n피고인\n【항 소 인】\n쌍방\n【검 사】\n홍영기 외 1인\n【변...,마약류관리에관한법률,원심판결을 파기한다.\n피고인을 징역 10월에 처한다.\n피고인에게 40시간의 약물...,10
85525,234621,도로교통법위반(음주운전),2023재고단3,20230418,선고,대구지법 포항지원,형사,판결 : 항소,음주운전 전력이 있는 피고인이 술에 취한 상태로 자동차를 운전하여 음주운전 금지규정...,음주운전 전력이 있는 피고인이 혈중알코올농도 0.177%의 술에 취한 상태로 자동차...,구 도로교통법(2023. 1. 3. 법률 제19158호로 개정되기 전의 것) 제44...,None,【피 고 인】\n피고인\n【검 사】\n김나연 외 1인\n【재심대상판결】\n대구지법 ...,도로교통법,피고인을 징역 10월에 처한다.\n다만 재심대상판결 확정일부터 2년간 위 형의 집행...,10


In [ ]:
#집행유예 추출
def extract_months_before_jiphaeng(text):
    # 정규 표현식을 사용하여 '집행' 이전의 '년간'과 숫자를 추출
    match = re.search(r'(\d+)\s*년\s*간.*집행', text)
    if match:
        return int(match.group(1))*12  # 소수점 버림 처리
    return None

In [ ]:
filtered_dataset_df['집행유예(개월)'] = filtered_dataset_df['판결'].apply(extract_months_before_jiphaeng)
filtered_dataset_df['집행유예(개월)'] = filtered_dataset_df['집행유예(개월)'].apply(lambda x: int(x) if isinstance(x, int) else x)


In [ ]:
#벌금 추출
def extract_fine_amount(text):
    # 정규 표현식을 사용하여 '원', '만 원', '환', 또는 '만 환' 앞의 금액을 추출
    match = re.search(r'(\d{1,3}(,\d{3})*)\s*원|(\d+)\s*만\s*원|(\d{1,3}(,\d{3})*)\s*환|(\d+)\s*만\s*환', text)
    if match:
        if match.group(1):  # 일반 '원' 단위
            amount = match.group(1).replace(',', '')
            return int(amount)
        elif match.group(3):  # '만 원' 단위
            amount = int(match.group(3)) * 10000
            return amount
    return None

In [ ]:
filtered_dataset_df['벌금']=filtered_dataset_df['판결'].apply(extract_fine_amount)

In [ ]:
filtered_dataset_df2=filtered_dataset_df[filtered_dataset_df['징역(개월)'].isna()&filtered_dataset_df['집행유예(개월)'].isna()&filtered_dataset_df['벌금'].isna()]
filtered_dataset_df2

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,참조조문,참조판례,전문,분류,판결,징역(개월),집행유예(개월),벌금
2,85835,부동산소유권이전등기,4281민상362,19480407,선고,대법원,민사,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증 나. 계약해제존재의 가항변에 대한 판...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,"민사소송법 제80조 , 제137조, 제395조",None,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",부동산,원판결을 파훼함.\n본건을 서울지방법원에 차려함.,None,NaN,NaN
7,85833,부동산이전등기말소,4281민상314,19490402,선고,대법원,민사,판결,부동산소유권이전 가등기권리자의 제3자에 대한 소유권이전 본등기의 말소청구권,가. 부동산소유권이전 가등기후에 등기의무자가 제3자에게 그 부동산소유권이전의 본등기...,부동산등기법 제2조,None,"【원고, 상고인】\n박희승\n【피고, 피상고인】\n조태환 외 32인\n【원심판결】\...",부동산,원판결을 파훼함.\n본건을 전주지방법원에 차려.,None,NaN,NaN
11,215267,국가보안법제3조급법령제19호제4조나항각위반피고,4283형상7,19501120,선고,대법원,형사,판결,긴급구속사건과 사법경찰관의 피의자신문조서 작성권의 존부,피의자를 긴급구속한 사법경찰관은 형사소송법의 개정 제8조에 의하여 그 구속을 유지하...,"형사소송법 제8조, 제56조, 제343조",None,"【피고인, 상고인】\n피고인\n【변 호 인】\n김성재\n【주 문】\n본건 상고를 기...",국가보안법,본건 상고를 기각한다.,None,NaN,NaN
12,85837,국가보안법위반,4283형상14,19501127,선고,대법원,형사,판결,국가보안법 제1조 제3호의 죄와 동법 제3조의 죄와의 죄질의 동일 여부,국가보안법 제1조 제3호의 결사는 국헌을 위배하여 정부를 잠칭하거나 국가를 변난할 ...,"국가보안법 제1조 제2항, 제3조 제2항 , 형법 제55조, 제45조",None,"【상고인, 피고인】\n【원심판결】\n제1심 부산지방법원, 제2심 동지방법원\n【주 ...",국가보안법,원판결을 파훼한다.\n본건을 대구고등법원에 이송한다.,None,NaN,NaN
13,85838,살인및살인미수,4283형상9,19501204,선고,대법원,형사,판결,소송기록의 분실과 판결의 기초되는 사실심리 및 증거조사와의 관계,기록이 분실되어 현존치 아니한 이상 원심판결은 그 기초되는 사실심리및 증거조사 없음...,None,None,"【상고인, 피고인】\n【원 심】\n대구고등법원\n【주 문】\n원판결을 파훼함.\n본...",살인,원판결을 파훼함.\n본건을 대구고등법원에 차려함.,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85545,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,"[2] 대법원 2020. 5. 14. 선고 2016다239024, 239031, 2...","【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",None,NaN,NaN
85551,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,[1] 민법 제109조 제1항 / [2] 민법 제109조 제1항 / [3] 민법 제...,"[1] 대법원 1955. 11. 10. 선고 4288민상321 판결, 대법원 199...","【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,None,NaN,NaN
85553,234719,근로에관한소송,2021다229588,20230427,선고,대법원,민사,판결,[1] 확인의 소에서 ‘확인의 이익’이 인정되기 위한 요건 [2] 사용사업주가 파견...,None,[1] 민사소송법 제250조 / [2] 구 파견근로자 보호 등에 관한 법률(2019...,[1][4] 대법원 2022. 7. 28. 선고 2016다40439 판결 / [1]...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1의 근로자지위확인청구 부분을 파기하고, 이 부분 제1심판결을 취...",None,NaN,NaN
85577,105970,"배임,사기",89도1309,19891128,선고,대법원,형사,판결,가. 동일한 피해자에 대한 3회의 금원편취 행위를 실체적 경합범으로 본 사례 나. ...,가. 피고인이 동일한 피해자로부터 3회에 걸쳐 돈을 편취함에 있어서 그 시간적 간격...,"가. 형법 제37조제347조 /나. 제355조 제2항, 민법 제372조",None,【피 고 인】\n【상 고 인】\n피고인\n【변 호 인】\n변호사 이정우\n【원심판결...,사기,상고를 기각한다.,None,NaN,NaN


In [ ]:
  def extract_text_between(text, start_patterns, end_pattern):
      for start_pattern in start_patterns:
          escaped_start_pattern = re.escape(start_pattern)
          escaped_end_pattern = re.escape(end_pattern)
          match = re.search(f'{escaped_start_pattern}(.*?){escaped_end_pattern}', text, re.DOTALL)
          if match:
              return match.group(1).strip()
      return None

In [ ]:
# 시작 패턴 리스트 정의
start_patterns = ['【원심판결】', '【원심결정】', '【환송판결】', '【제1심판결】', '【제1심결정】']
filtered_dataset_df2['원심결과'] = filtered_dataset_df2['전문'].apply(lambda x: extract_text_between(x, start_patterns, '【주 문】'))

<ipython-input-72-e6c12596134e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset_df2['원심결과'] = filtered_dataset_df2['전문'].apply(lambda x: extract_text_between(x, start_patterns, '【주 문】'))


In [ ]:
filtered_dataset_df2

In [ ]:
# 원심결과가 결측치인 행 제거
filtered_dataset_df2_cleaned = filtered_dataset_df2.dropna(subset=['원심결과'])

In [ ]:
pd.reset_option('display.max_colwidth')

In [ ]:
filtered_dataset_df2_cleaned[filtered_dataset_df2_cleaned['사건명'].str.contains('절도')]

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,참조조문,참조판례,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과
142,215305,절도피고,4287형상135,19541005,선고,대법원,형사,판결,증거취사의 한계와 심리부진,범죄사실 유무의 판단에 영향이 미칠 만한 증거와 사실을 무시 간과하고 만연히 범죄의...,구형사소송법 제337조,None,"【상 고 인】\n검사 주재황\n【원심판결】\n제1심 대전지방법원 서산지원, 제2심 ...",절도,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,"제1심 대전지방법원 서산지원, 제2심 서울고등법원"
223,215245,건조물침입절도,4287형상100,19550304,선고,대법원,형사,판결,사실오인과 의률착오,"업무상 횡령에 해당하는 사실을 건조물 침입, 절도로 오인하여 당해 법조를 적용하였음...","형법 제330조, 제355조, 제356조",None,"【상고인, 피고인】\n피고인\n【원심판결】\n제1심 전주지방법원 군산지원, 제2심 ...",절도,원판결을 파기하고 본건을 광주고등법원에 환송한다.,None,NaN,NaN,"제1심 전주지방법원 군산지원, 제2심 광주고등법원"
233,215239,"병역법위반,절도등",4287형상89,19550401,선고,대법원,형사,판결,범죄성립 조각 사유의 주장과 판단유탈,범죄성립 조각사유의 주장을 판단하지 아니함은 위법이다.,구 형사소송법 제360조,None,"【상고인, 피고인】\n피고인\n【피고대리인】\n변호사 방준향\n【원심판결】\n제1심...",절도,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,"제1심 서울지방법원 인천지원, 제2심 서울고등법원"
276,86058,국가보안법위반및살인사체유기주거침입절도피고,4288형상81,19550614,선고,대법원,형사,판결,사실인정에 인용한 증거일부의 부적성과 판결의 정부,범죄사실의 인정에 인용한 증거의 일부가 증거법칙에 위반됨이 있다 할지라도 이를 제외...,"구 형사소송법 제340조 제1항, 제341조, 제410조 제1항, 제410조 제13...",None,"【상고인, 피고인】\n【원심판결】\n제1심 대전지방법원, 제2심 서울고등법원\n【주...",살인,본건상고를 기각한다,None,NaN,NaN,"제1심 대전지방법원, 제2심 서울고등법원"
361,86160,절도,4288형상280,19551015,선고,대법원,형사,판결,피해자의 추상적 진술과 채증법칙의 적부,피해자의 진술내용이 범행의 일부에 대한 추상적임에 불과한 것임에 불구하고 이 공술을...,"형사소송법 제307조, 제308조",None,"【상고인, 피고인】\n【변 호 인】\n김용달\n【원심판결】\n제1심 서울지방법원, ...",절도,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84757,232091,성폭력범죄의처벌등에관한특례법위반(주거침입강제추행)·준강제추행·절도미수·성폭력범죄의처...,"2022도1272, 2022전도9",20220414,선고,대법원,형사,판결,[1] 주거침입죄의 보호법익(=사실상 주거의 평온) / 주거침입죄의 구성요건적 행위...,None,[1] 형법 제319조 제1항 / [2] 형법 제319조 제1항,[1] 대법원 2021. 9. 9. 선고 2020도12630 전원합의체 판결(공20...,【피고인 겸 피부착명령청구자】\n피고인\n【상 고 인】\n피고인 겸 피부착명령청구자...,절도,상고를 모두 기각한다.,None,NaN,NaN,수원고법 2022. 1. 12. 선고 2021노535 판결
84771,232093,살인·절도·특수주거침입·정보통신망이용촉진및정보보호등에관한법률위반(정보통신망침해등)·...,"2022도1662, 2022전도15",20220414,선고,대법원,형사,판결,온라인 게임을 통해 알게 된 피해자와 전화통화를 하고 단 몇 차례 만나게 된 것을 ...,None,형사소송법상 제383조 제4호,None,【피고인 겸 피부착명령청구자】\n피고인\n【상 고 인】\n피고인 겸 피부착명령청구자...,살인,상고를 모두 기각한다.,None,NaN,NaN,"서울고법 2022. 1. 19. 선고 2021노2032, 2021전노150 판결"
85095,232909,성폭력범죄의처벌등에관한특례법위반(통신매체이용음란)·야간건조물침입절도,2022도5659,20220728,선고,대법원,형사,판결,"[1] 원심판결에 법리오해 등의 위법이 있다는 주장이 적법한 상고이유가 아닌 경우,...",None,"[1] 형사소송법 제383조 제1호, 제2호, 제3호, 제384조 / [2] 형법 ...","[1] 대법원 2010. 1. 28. 선고 2009도11407 판결, 대법원 201...",【피 고 인】\n피고인\n【상 고 인】\n피고인\n【변 호 인】\n변호사 유익상\n...,절도,"원심판결을 파기하고, 사건을 전주지방법원에 환송한다.",None,NaN,NaN,전주지법 2022. 4. 27. 선고 2022노72 판결
85102,227727,특정범죄가중처벌등에관한법률위반(절도),2020도13705,20220728,선고,대법원,형사,판결,징역형의 집행유예를 선고한 판결이 확정된 후 선고의 실효 또는 취소 없이 유예기간을...,특정범죄 가중처벌 등에 관한 법률(이하 ‘특정범죄가중법’이라 한다) 제5조의4 제5...,"특정범죄 가중처벌 등에 관한 법률 제5조의4 제5항 제1호, 형법 제65조, 제32...","대법원 2010. 9. 9. 선고 2010도8021 판결(공2010하, 1963),...",【피 고 인】\n피고인\n【상 고 인】\n피고인\n【변 호 인】\n변호사 김지후\n...,절도,"원심판결을 파기하고, 사건을 대전지방법원에 환송한다.",None,NaN,NaN,대전지법 2020. 9. 17. 선고 2020노1999 판결


In [ ]:
pd.set_option('display.max_colwidth', None)
filtered_dataset_df2_cleaned['원심결과']

2               서울지방법원 1948. 4. 1 선고 47민185(1심, 비약상고) 판결
7            청주지방법원 1948. 1. 9 선고 46민46, 78(1심, 비약상고) 판결
12                                 제1심 부산지방법원, 제2심 동지방법원
26                      서울고등법원 1952. 3. 25 선고 50민공316 판결
29       제1심 부산지방법원, 제2심 대구고등법원 1952. 7. 30 선고 52민공80 판결
                              ...                       
85545               서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결
85551                서울고법 2017. 4. 7. 선고 2015나2055371 판결
85553               서울고법 2021. 4. 9. 선고 (춘천)2020나1078 판결
85577            대구지방법원 1989.5.25. 선고 87노1993, 88노663 판결
85639                     대구고등법원 1991.9.11. 선고 91노395 판결
Name: 원심결과, Length: 13538, dtype: object

In [ ]:
# '선고'와 '판결' 사이의 텍스트 추출
def extract_sentece_number(text):
    match = re.search(r'선고\s+(.*?)\s+판결', text)
    if match:
        content = match.group(1)
        # 괄호가 있는 경우 처리
        if '(' in content:
            parts = content.split('(')
            # 숫자 뒤에 괄호가 있는 경우, 괄호 전까지 추출
            if re.search(r'\d', parts[0]):
                return parts[0].strip()
            content = re.split(r'\)\s*', content)[-1].strip()
        return content.strip()
    return None

In [ ]:
filtered_dataset_df2_cleaned['원심번호'] = filtered_dataset_df2_cleaned['원심결과'].apply(extract_sentece_number)

<ipython-input-76-427a1390e337>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset_df2_cleaned['원심번호'] = filtered_dataset_df2_cleaned['원심결과'].apply(extract_sentece_number)


In [ ]:
len(filtered_dataset_df2_cleaned)

13538

In [ ]:
pd.reset_option('display.max_colwidth')
filtered_dataset_df2_cleaned= filtered_dataset_df2_cleaned[filtered_dataset_df2_cleaned['원심번호'].notna()]
filtered_dataset_df2_cleaned['원심번호'] = filtered_dataset_df2_cleaned['원심번호'].apply(lambda x: x.split(',')[0])

<ipython-input-77-d25c09d45e92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset_df2_cleaned['원심번호'] = filtered_dataset_df2_cleaned['원심번호'].apply(lambda x: x.split(',')[0])


In [ ]:
len(filtered_dataset_df2_cleaned)

12484

In [ ]:
pd.reset_option('display.max_colwidth')
filtered_dataset_df2_cleaned

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,날짜,URL,원심번호
2,85835,부동산소유권이전등기,4281민상362,19480407,선고,대법원,민사,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증 나. 계약해제존재의 가항변에 대한 판...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,...,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",부동산,원판결을 파훼함.\n본건을 서울지방법원에 차려함.,None,NaN,NaN,"서울지방법원 1948. 4. 1 선고 47민185(1심, 비약상고) 판결",1948.04.01.,None,47민185
7,85833,부동산이전등기말소,4281민상314,19490402,선고,대법원,민사,판결,부동산소유권이전 가등기권리자의 제3자에 대한 소유권이전 본등기의 말소청구권,가. 부동산소유권이전 가등기후에 등기의무자가 제3자에게 그 부동산소유권이전의 본등기...,...,"【원고, 상고인】\n박희승\n【피고, 피상고인】\n조태환 외 32인\n【원심판결】\...",부동산,원판결을 파훼함.\n본건을 전주지방법원에 차려.,None,NaN,NaN,"청주지방법원 1948. 1. 9 선고 46민46, 78(1심, 비약상고) 판결",1948.01.09.,None,46민46
26,85866,부동산소유권확인,4285민상111,19520131,선고,대법원,민사,판결,가. 사문서이 증거력과 자백 나. 귀속농지사건에 관한 사문서의 증거력과 자백,가. 사문서의 성립에 당사자간 다툼이 없으면 이는 상대방이 그 서증의 진정성립에 관...,...,"【원고, 피공소인】\n김춘자\n【피고, 공소인, 상고인】\n국 소송대리인 남성우\n...",부동산,원판결을 파훼한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,서울고등법원 1952. 3. 25 선고 50민공316 판결,1952.03.25.,None,50민공316
29,85899,부동산소유권이전등기절차이행급가옥명도,4285민상122,19520221,선고,대법원,민사,판결,부재자의 소유부동산에 대한 그 처의 처분권 유무,부의 부재중 그 처가 부의 소유부동산에 대하여 관리는 유효히 행할 수 있을 지언정 ...,...,"【원고, 상고인】\n김용주 소송대리인변호사 장후영\n【피고, 피상고인】\n윤복수 외...",부동산,본건 상고를 기각한다.\n상고비용은 원고의 부담으로 한다.,None,NaN,NaN,"제1심 부산지방법원, 제2심 대구고등법원 1952. 7. 30 선고 52민공80 판결",1952.07.30.,None,52민공80
33,85905,부동산소유권확인,4285민상187,19520312,선고,대법원,민사,판결,선행된 매매와 대물변제의 주장에 대한 심리의 순서,원고는 갑이 을의 소유부동산을 매수하여 원고에 대한 채무의 대물변제에 공하였다 주장...,...,"【원고, 피상고인】\n이성환\n【피고, 상고인】\n국 대표자 관재청장 유완창\n【원...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1952. 5. 29 선고 49민공427 판결",1952.05.29.,None,49민공427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85545,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,...,"【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",None,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결,2021.04.09.,None,2020나1108
85551,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,...,"【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,None,NaN,NaN,서울고법 2017. 4. 7. 선고 2015나2055371 판결,2017.04.07.,None,2015나2055371
85553,234719,근로에관한소송,2021다229588,20230427,선고,대법원,민사,판결,[1] 확인의 소에서 ‘확인의 이익’이 인정되기 위한 요건 [2] 사용사업주가 파견...,None,...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1의 근로자지위확인청구 부분을 파기하고, 이 부분 제1심판결을 취...",None,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1078 판결,2021.04.09.,None,2020나1078
85577,105970,"배임,사기",89도1309,19891128,선고,대법원,형사,판결,가. 동일한 피해자에 대한 3회의 금원편취 행위를 실체적 경합범으로 본 사례 나. ...,가. 피고인이 동일한 피해자로부터 3회에 걸쳐 돈을 편취함에 있어서 그 시간적 간격...,...,【피 고 인】\n【상 고 인】\n피고인\n【변 호 인】\n변호사 이정우\n【원심판결...,사기,상고를 기각한다.,None,NaN,NaN,"대구지방법원 1989.5.25. 선고 87노1993, 88노663 판결",None,None,87노1993


In [ ]:
import re

def extract_and_format_date(text):
    if isinstance(text, str):
        match = re.search(r'(\d{4})\. ?(\d{1,2})\. ?(\d{1,2})\.?', text)
        if match:
            year = match.group(1)
            month = match.group(2).zfill(2)
            day = match.group(3).zfill(2)
            return f"{year}.{month}.{day}."
    return None

# '원심결과' 열에서 날짜 변환
filtered_dataset_df2_cleaned['날짜'] = filtered_dataset_df2_cleaned['원심결과'].apply(extract_and_format_date)
filtered_dataset_df2_cleaned


,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,참조판례,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,원심번호,날짜
2,85835,부동산소유권이전등기,4281민상362,19480407,선고,대법원,민사,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증 나. 계약해제존재의 가항변에 대한 판...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,...,None,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",부동산,원판결을 파훼함.\n본건을 서울지방법원에 차려함.,None,NaN,NaN,"서울지방법원 1948. 4. 1 선고 47민185(1심, 비약상고) 판결",47민185,1948.04.01.
7,85833,부동산이전등기말소,4281민상314,19490402,선고,대법원,민사,판결,부동산소유권이전 가등기권리자의 제3자에 대한 소유권이전 본등기의 말소청구권,가. 부동산소유권이전 가등기후에 등기의무자가 제3자에게 그 부동산소유권이전의 본등기...,...,None,"【원고, 상고인】\n박희승\n【피고, 피상고인】\n조태환 외 32인\n【원심판결】\...",부동산,원판결을 파훼함.\n본건을 전주지방법원에 차려.,None,NaN,NaN,"청주지방법원 1948. 1. 9 선고 46민46, 78(1심, 비약상고) 판결",46민46,1948.01.09.
26,85866,부동산소유권확인,4285민상111,19520131,선고,대법원,민사,판결,가. 사문서이 증거력과 자백 나. 귀속농지사건에 관한 사문서의 증거력과 자백,가. 사문서의 성립에 당사자간 다툼이 없으면 이는 상대방이 그 서증의 진정성립에 관...,...,None,"【원고, 피공소인】\n김춘자\n【피고, 공소인, 상고인】\n국 소송대리인 남성우\n...",부동산,원판결을 파훼한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,서울고등법원 1952. 3. 25 선고 50민공316 판결,50민공316,1952.03.25.
29,85899,부동산소유권이전등기절차이행급가옥명도,4285민상122,19520221,선고,대법원,민사,판결,부재자의 소유부동산에 대한 그 처의 처분권 유무,부의 부재중 그 처가 부의 소유부동산에 대하여 관리는 유효히 행할 수 있을 지언정 ...,...,None,"【원고, 상고인】\n김용주 소송대리인변호사 장후영\n【피고, 피상고인】\n윤복수 외...",부동산,본건 상고를 기각한다.\n상고비용은 원고의 부담으로 한다.,None,NaN,NaN,"제1심 부산지방법원, 제2심 대구고등법원 1952. 7. 30 선고 52민공80 판결",52민공80,1952.07.30.
33,85905,부동산소유권확인,4285민상187,19520312,선고,대법원,민사,판결,선행된 매매와 대물변제의 주장에 대한 심리의 순서,원고는 갑이 을의 소유부동산을 매수하여 원고에 대한 채무의 대물변제에 공하였다 주장...,...,None,"【원고, 피상고인】\n이성환\n【피고, 상고인】\n국 대표자 관재청장 유완창\n【원...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1952. 5. 29 선고 49민공427 판결",49민공427,1952.05.29.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85545,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,...,"[2] 대법원 2020. 5. 14. 선고 2016다239024, 239031, 2...","【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",None,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결,2020나1108,2021.04.09.
85551,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,...,"[1] 대법원 1955. 11. 10. 선고 4288민상321 판결, 대법원 199...","【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,None,NaN,NaN,서울고법 2017. 4. 7. 선고 2015나2055371 판결,2015나2055371,2017.04.07.
85553,234719,근로에관한소송,2021다229588,20230427,선고,대법원,민사,판결,[1] 확인의 소에서 ‘확인의 이익’이 인정되기 위한 요건 [2] 사용사업주가 파견...,None,...,[1][4] 대법원 2022. 7. 28. 선고 2016다40439 판결 / [1]...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1의 근로자지위확인청구 부분을 파기하고, 이 부분 제1심판결을 취...",None,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1078 판결,2020나1078,2021.04.09.
85577,105970,"배임,사기",89도1309,19891128,선고,대법원,형사,판결,가. 동일한 피해자에 대한 3회의 금원편취 행위를 실체적 경합범으로 본 사례 나. ...,가. 피고인이 동일한 피해자로부터 3회에 걸쳐 돈을 편취함에 있어서 그 시간적 간격...,...,None,【피 고 인】\n【상 고 인】\n피고인\n【변 호 인】\n변호사 이정우\n【원심판결...,사기,상고를 기각한다.,None,NaN,NaN,"대구지방법원 1989.5.25. 선고 87노1993, 88노663 판결",87노1993,1989.05.25.


In [ ]:
filtered_dataset_df2_cleaned.loc[filtered_dataset_df2_cleaned['판례정보일련번호'] == 102742, '날짜'] = "1984.05.23."
filtered_dataset_df2_cleaned.loc[filtered_dataset_df2_cleaned['판례정보일련번호'] == 105536, '날짜'] = "1989.05.25."
filtered_dataset_df2_cleaned.loc[filtered_dataset_df2_cleaned['판례정보일련번호'] == 189675, '날짜'] = "1991.12.00."
filtered_dataset_df2_cleaned.loc[filtered_dataset_df2_cleaned['판례정보일련번호'] == 212169, '날짜'] = "2001.11.07."


In [ ]:
filtered_dataset_df2_cleaned[filtered_dataset_df2_cleaned['날짜'].isna()]

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,날짜,URL,원심번호


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.law.go.kr/',
    'Connection': 'keep-alive',
    'DNT': '1',  # Do Not Track 요청
}

# 세션 사용
session = requests.Session()
session.headers.update(headers)

nofound = 0
# 새로운 열을 추가할 준비
filtered_dataset_df2_cleaned['URL'] = None

# URL 처리 및 데이터 추출
for index, row in tqdm(filtered_dataset_df2_cleaned.iterrows(), total=filtered_dataset_df2_cleaned.shape[0], desc="Processing cases"):
    case_number = row['원심번호']
    target_date = row['날짜']
    target_year = target_date.split('.')[0]  # 연도만 추출
    url = f"https://www.law.go.kr/DRF/lawSearch.do?OC=spwldmlgla&target=prec&type=HTML&nb={case_number}"

    # 웹페이지 요청
    response = session.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # <a> 태그 추출 (href가 'javascript:;' 또는 '/DRF/lawService.do?'로 시작된 태그들 찾기)
        a_tags = soup.find_all('a', href=True)
        urls = []

        for a_tag in a_tags:
            href = a_tag['href']
            # 날짜가 포함된 tr 찾기
            tr = a_tag.find_parent('tr')
            if tr:
                tds = tr.find_all('td')
                if len(tds) >= 6:  # 날짜가 포함된 td가 있는지 확인
                    date_td = tds[5].get_text(strip=True)
                    date_year = date_td.split('.')[0]

                    if date_year == target_year:
                        if href.startswith('javascript:;'):
                            onclick = a_tag.get('onclick', '')
                            # onclick에서 ID 추출
                            id_start = onclick.find("'") + 1
                            id_end = onclick.find("'", id_start)
                            extracted_id = onclick[id_start:id_end]

                            # 실제 URL 구성
                            full_url = f"https://www.law.go.kr/DRF/lawService.do?OC=spwldmlgla&target=prec&ID={extracted_id}&type=HTML"
                            urls.append(full_url)
                        elif href.startswith('/DRF/lawService.do?'):
                            full_url = f"https://www.law.go.kr{href}"
                            urls.append(full_url)

        # DataFrame의 해당 행에 URL 리스트 추가
        filtered_dataset_df2_cleaned.at[index, 'URL'] = urls if urls else None
    else:
        print(f"Failed to retrieve the page for case number {case_number}. Status code: {response.status_code}")

# 결과 출력
filtered_dataset_df2_cleaned


<ipython-input-52-fe87813b82ac>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataset_df2_cleaned['URL'] = None
Processing cases: 100%|██████████| 12484/12484 [46:36<00:00,  4.46it/s]


,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,날짜,URL,원심번호
2,85835,부동산소유권이전등기,4281민상362,19480407,선고,대법원,민사,선고,가. 소송위임 흠결의 항변과 당해 이원의 인증 나. 계약해제존재의 가항변에 대한 판...,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,...,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",부동산,원판결을 파훼함.\n본건을 서울지방법원에 차려함.,None,NaN,NaN,"서울지방법원 1948. 4. 1 선고 47민185(1심, 비약상고) 판결",1948.04.01.,None,47민185
7,85833,부동산이전등기말소,4281민상314,19490402,선고,대법원,민사,판결,부동산소유권이전 가등기권리자의 제3자에 대한 소유권이전 본등기의 말소청구권,가. 부동산소유권이전 가등기후에 등기의무자가 제3자에게 그 부동산소유권이전의 본등기...,...,"【원고, 상고인】\n박희승\n【피고, 피상고인】\n조태환 외 32인\n【원심판결】\...",부동산,원판결을 파훼함.\n본건을 전주지방법원에 차려.,None,NaN,NaN,"청주지방법원 1948. 1. 9 선고 46민46, 78(1심, 비약상고) 판결",1948.01.09.,None,46민46
26,85866,부동산소유권확인,4285민상111,19520131,선고,대법원,민사,판결,가. 사문서이 증거력과 자백 나. 귀속농지사건에 관한 사문서의 증거력과 자백,가. 사문서의 성립에 당사자간 다툼이 없으면 이는 상대방이 그 서증의 진정성립에 관...,...,"【원고, 피공소인】\n김춘자\n【피고, 공소인, 상고인】\n국 소송대리인 남성우\n...",부동산,원판결을 파훼한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,서울고등법원 1952. 3. 25 선고 50민공316 판결,1952.03.25.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,50민공316
29,85899,부동산소유권이전등기절차이행급가옥명도,4285민상122,19520221,선고,대법원,민사,판결,부재자의 소유부동산에 대한 그 처의 처분권 유무,부의 부재중 그 처가 부의 소유부동산에 대하여 관리는 유효히 행할 수 있을 지언정 ...,...,"【원고, 상고인】\n김용주 소송대리인변호사 장후영\n【피고, 피상고인】\n윤복수 외...",부동산,본건 상고를 기각한다.\n상고비용은 원고의 부담으로 한다.,None,NaN,NaN,"제1심 부산지방법원, 제2심 대구고등법원 1952. 7. 30 선고 52민공80 판결",1952.07.30.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,52민공80
33,85905,부동산소유권확인,4285민상187,19520312,선고,대법원,민사,판결,선행된 매매와 대물변제의 주장에 대한 심리의 순서,원고는 갑이 을의 소유부동산을 매수하여 원고에 대한 채무의 대물변제에 공하였다 주장...,...,"【원고, 피상고인】\n이성환\n【피고, 상고인】\n국 대표자 관재청장 유완창\n【원...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,None,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1952. 5. 29 선고 49민공427 판결",1952.05.29.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,49민공427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85545,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,...,"【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",None,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결,2021.04.09.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2020나1108
85551,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,...,"【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,None,NaN,NaN,서울고법 2017. 4. 7. 선고 2015나2055371 판결,2017.04.07.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2015나2055371
85553,234719,근로에관한소송,2021다229588,20230427,선고,대법원,민사,판결,[1] 확인의 소에서 ‘확인의 이익’이 인정되기 위한 요건 [2] 사용사업주가 파견...,None,...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1의 근로자지위확인청구 부분을 파기하고, 이 부분 제1심판결을 취...",None,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1078 판결,2021.04.09.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2020나1078
85577,105970,"배임,사기",89도1309,19891128,선고,대법원,형사,판결,가. 동일한 피해자에 대한 3회의 금원편취 행위를 실체적 경합범으로 본 사례 나. ...,가. 피고인이 동일한 피해자로부터 3회에 걸쳐 돈을 편취함에 있어서 그 시간적 간격...,...,【피 고 인】\n【상 고 인】\n피고인\n【변 호 인】\n변호사 이정우\n【원심판결...,사기,상고를 기각한다.,None,NaN,NaN,"대구지방법원 1989.5.25. 선고 87노1993, 88노663 판결",1989.05.25.,None,87노1993


In [ ]:
# 'URL' 열에서 결측치를 가진 행을 삭제
filtered_dataset_df2_cleaned = filtered_dataset_df2_cleaned.dropna(subset=['URL'])

In [ ]:
import tarfile
filtered_dataset_df2_cleaned.to_csv('filtered_dataset_df2_cleaned.csv', index=False)
with tarfile.open('filtered_dataset_df2_cleaned.tar.gz', 'w:gz') as tar:
    tar.add('filtered_dataset_df2_cleaned.csv', arcname='filtered_dataset_df2_cleaned.csv')

In [ ]:
import tarfile
import pandas as pd
import os

# tar.gz 파일 경로
tar_gz_path = 'filtered_dataset_df2_cleaned.tar.gz'

# tar.gz 파일 압축 해제
with tarfile.open(tar_gz_path, 'r:gz') as tar:
    tar.extractall(path='/content')

# 압축 해제된 CSV 파일 경로
csv_path = 'filtered_dataset_df2_cleaned.csv'

# CSV 파일을 데이터프레임으로 읽기
filtered_dataset_df2_cleaned = pd.read_csv(csv_path)

# 데이터프레임 출력
filtered_dataset_df2_cleaned


,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,날짜,URL,원심번호
0,85866,부동산소유권확인,4285민상111,19520131,선고,대법원,민사,판결,가. 사문서이 증거력과 자백 나. 귀속농지사건에 관한 사문서의 증거력과 자백,가. 사문서의 성립에 당사자간 다툼이 없으면 이는 상대방이 그 서증의 진정성립에 관...,...,"【원고, 피공소인】\n김춘자\n【피고, 공소인, 상고인】\n국 소송대리인 남성우\n...",부동산,원판결을 파훼한다.\n본건을 서울고등법원에 환송한다.,NaN,NaN,NaN,서울고등법원 1952. 3. 25 선고 50민공316 판결,1952.03.25.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,50민공316
1,85899,부동산소유권이전등기절차이행급가옥명도,4285민상122,19520221,선고,대법원,민사,판결,부재자의 소유부동산에 대한 그 처의 처분권 유무,부의 부재중 그 처가 부의 소유부동산에 대하여 관리는 유효히 행할 수 있을 지언정 ...,...,"【원고, 상고인】\n김용주 소송대리인변호사 장후영\n【피고, 피상고인】\n윤복수 외...",부동산,본건 상고를 기각한다.\n상고비용은 원고의 부담으로 한다.,NaN,NaN,NaN,"제1심 부산지방법원, 제2심 대구고등법원 1952. 7. 30 선고 52민공80 판결",1952.07.30.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,52민공80
2,85905,부동산소유권확인,4285민상187,19520312,선고,대법원,민사,판결,선행된 매매와 대물변제의 주장에 대한 심리의 순서,원고는 갑이 을의 소유부동산을 매수하여 원고에 대한 채무의 대물변제에 공하였다 주장...,...,"【원고, 피상고인】\n이성환\n【피고, 상고인】\n국 대표자 관재청장 유완창\n【원...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,NaN,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1952. 5. 29 선고 49민공427 판결",1952.05.29.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,49민공427
3,85863,부동산소유권이전등기절차이행,4286민상10,19530627,선고,대법원,민사,판결,민사소송법 제35조 제6호 법의,판사가 일심에서 단지 구두변론 증거조사 증거결정 등에 입회관여하였을 뿐이고 종국판결...,...,"【원고, 피공소인, 상고인】\n박덕영 소송대리인 변호인 정구영\n【피고, 공소인, ...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,NaN,NaN,NaN,서울고등법원 1952. 10. 30 선고 52민공80 판결,1952.10.30.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,52민공80
4,215311,부동산소유권이전등기말소등기절차이행,4287민상304,19540313,선고,대법원,민사,판결,사실상 법원의 증거취사와 사실인정의 권한,증거의 취사사실의 인정은 사실심법원의 전권에 속한 사항으로서 실험칙 또는 논리 등에...,...,"【원고, 상고인】\n원고 소송대리인 변호사 지문식\n【피고, 피상고인】\n피고 1외...",부동산,본건 상고를 기각한다\n상고소송비용은 원고의 부담으로 한다,NaN,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1954. 3. 30 선고 53민공202 판결",1954.03.30.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,53민공202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,234711,노동조합및노동관계조정법위반·근로기준법위반·남녀고용평등과일·가정양립지원에관한법률위반,2020도16431,20230427,선고,대법원,형사,판결,[1] 구 근로기준법 제51조 제1항에서 정한 탄력적 근로시간제는 취업규칙에 의하여...,[1] 구 근로기준법(2017. 11. 28. 법률 제15108호로 개정되기 전의 ...,...,【피 고 인】\n피고인\n【상 고 인】\n피고인 및 검사\n【변 호 인】\n변호사 ...,근로기준법,원심판결 중 유죄 부분(이유무죄 부분 포함)과 연장근로수당 미지급으로 인한 근로기준...,NaN,NaN,NaN,인천지법 2020. 11. 5. 선고 2019노3882 판결,2020.11.05.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,2019노3882
5004,234703,마약류관리에관한법률위반(향정),2023도2102,20230427,선고,대법원,형사,판결,[1] 형사소송법 제254조 제4항에서 공소사실의 특정을 요구하는 취지 및 범죄 ‘...,"[1] 공소사실의 기재는 범죄의 일시, 장소와 방법을 명시하여 사실을 특정할 수 있...",...,【피 고 인】\n피고인\n【상 고 인】\n피고인\n【변 호 인】\n변호사 윤대진\n...,마약류관리에관한법률,"원심판결을 파기하고, 사건을 대구지방법원에 환송한다.",NaN,NaN,NaN,대구지법 2023. 1. 18. 선고 2022노3872 판결,2023.01.18.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,2022노3872
5005,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,...,"【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",NaN,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결,2021.04.09.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,2020나1108
5006,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,...,"【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,NaN,NaN,NaN,서울고법 2017. 4. 7. 선고 2015나2055371 판결,2017.04.07.,['https://www.law.go.kr/DRF/lawService.do?OC=s...,2015나2055371


In [ ]:
import re

# Function to ensure URLs have a scheme
def ensure_url_scheme(url):
    if not re.match(r'^https?://', url):
        return 'https://' + url
    return url

# Apply the function to the URL column
filtered_dataset_df2_cleaned['URL'] = filtered_dataset_df2_cleaned['URL'].apply(
    lambda urls: [ensure_url_scheme(url) for url in ast.literal_eval(urls)]
)


In [ ]:
filtered_dataset_df2_cleaned

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,날짜,URL,원심번호
0,85866,부동산소유권확인,4285민상111,19520131,선고,대법원,민사,판결,가. 사문서이 증거력과 자백 나. 귀속농지사건에 관한 사문서의 증거력과 자백,가. 사문서의 성립에 당사자간 다툼이 없으면 이는 상대방이 그 서증의 진정성립에 관...,...,"【원고, 피공소인】\n김춘자\n【피고, 공소인, 상고인】\n국 소송대리인 남성우\n...",부동산,원판결을 파훼한다.\n본건을 서울고등법원에 환송한다.,NaN,NaN,NaN,서울고등법원 1952. 3. 25 선고 50민공316 판결,1952.03.25.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,50민공316
1,85899,부동산소유권이전등기절차이행급가옥명도,4285민상122,19520221,선고,대법원,민사,판결,부재자의 소유부동산에 대한 그 처의 처분권 유무,부의 부재중 그 처가 부의 소유부동산에 대하여 관리는 유효히 행할 수 있을 지언정 ...,...,"【원고, 상고인】\n김용주 소송대리인변호사 장후영\n【피고, 피상고인】\n윤복수 외...",부동산,본건 상고를 기각한다.\n상고비용은 원고의 부담으로 한다.,NaN,NaN,NaN,"제1심 부산지방법원, 제2심 대구고등법원 1952. 7. 30 선고 52민공80 판결",1952.07.30.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,52민공80
2,85905,부동산소유권확인,4285민상187,19520312,선고,대법원,민사,판결,선행된 매매와 대물변제의 주장에 대한 심리의 순서,원고는 갑이 을의 소유부동산을 매수하여 원고에 대한 채무의 대물변제에 공하였다 주장...,...,"【원고, 피상고인】\n이성환\n【피고, 상고인】\n국 대표자 관재청장 유완창\n【원...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,NaN,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1952. 5. 29 선고 49민공427 판결",1952.05.29.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,49민공427
3,85863,부동산소유권이전등기절차이행,4286민상10,19530627,선고,대법원,민사,판결,민사소송법 제35조 제6호 법의,판사가 일심에서 단지 구두변론 증거조사 증거결정 등에 입회관여하였을 뿐이고 종국판결...,...,"【원고, 피공소인, 상고인】\n박덕영 소송대리인 변호인 정구영\n【피고, 공소인, ...",부동산,원판결을 파기한다.\n본건을 서울고등법원에 환송한다.,NaN,NaN,NaN,서울고등법원 1952. 10. 30 선고 52민공80 판결,1952.10.30.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,52민공80
4,215311,부동산소유권이전등기말소등기절차이행,4287민상304,19540313,선고,대법원,민사,판결,사실상 법원의 증거취사와 사실인정의 권한,증거의 취사사실의 인정은 사실심법원의 전권에 속한 사항으로서 실험칙 또는 논리 등에...,...,"【원고, 상고인】\n원고 소송대리인 변호사 지문식\n【피고, 피상고인】\n피고 1외...",부동산,본건 상고를 기각한다\n상고소송비용은 원고의 부담으로 한다,NaN,NaN,NaN,"제1심 서울지방법원, 제2심 서울고등법원 1954. 3. 30 선고 53민공202 판결",1954.03.30.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,53민공202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,234711,노동조합및노동관계조정법위반·근로기준법위반·남녀고용평등과일·가정양립지원에관한법률위반,2020도16431,20230427,선고,대법원,형사,판결,[1] 구 근로기준법 제51조 제1항에서 정한 탄력적 근로시간제는 취업규칙에 의하여...,[1] 구 근로기준법(2017. 11. 28. 법률 제15108호로 개정되기 전의 ...,...,【피 고 인】\n피고인\n【상 고 인】\n피고인 및 검사\n【변 호 인】\n변호사 ...,근로기준법,원심판결 중 유죄 부분(이유무죄 부분 포함)과 연장근로수당 미지급으로 인한 근로기준...,NaN,NaN,NaN,인천지법 2020. 11. 5. 선고 2019노3882 판결,2020.11.05.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2019노3882
5004,234703,마약류관리에관한법률위반(향정),2023도2102,20230427,선고,대법원,형사,판결,[1] 형사소송법 제254조 제4항에서 공소사실의 특정을 요구하는 취지 및 범죄 ‘...,"[1] 공소사실의 기재는 범죄의 일시, 장소와 방법을 명시하여 사실을 특정할 수 있...",...,【피 고 인】\n피고인\n【상 고 인】\n피고인\n【변 호 인】\n변호사 윤대진\n...,마약류관리에관한법률,"원심판결을 파기하고, 사건을 대구지방법원에 환송한다.",NaN,NaN,NaN,대구지법 2023. 1. 18. 선고 2022노3872 판결,2023.01.18.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2022노3872
5005,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,...,"【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",NaN,NaN,NaN,서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결,2021.04.09.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2020나1108
5006,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,...,"【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,NaN,NaN,NaN,서울고법 2017. 4. 7. 선고 2015나2055371 판결,2017.04.07.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2015나2055371


# URL 접속하여 원심판결에서 형량 추출

In [ ]:
import ast
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.law.go.kr/',
    'Connection': 'keep-alive',
    'DNT': '1',  # Do Not Track 요청
}

# 세션 사용
session = requests.Session()
session.headers.update(headers)
found=0
# URL 처리 및 데이터 추출
for index, row in tqdm(filtered_dataset_df2_cleaned.iterrows(), total=filtered_dataset_df2_cleaned.shape[0], desc="Processing cases"):
    urls = row['URL']
    for url in urls:
        try:
            response = session.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # iframe src 추출
                iframe = soup.find('iframe')
                if iframe and 'src' in iframe.attrs:
                    iframe_src = iframe['src']
                    iframe_response = session.get(iframe_src)
                    if iframe_response.status_code == 200:
                        iframe_soup = BeautifulSoup(iframe_response.content, 'html.parser')
                        text = iframe_soup.get_text()

                        # 주 문과 이 유 사이의 텍스트 추출
                        result = re.search(r'【주 문】(.*?)【이 유】', text, re.DOTALL)
                        if result:
                            content = result.group(1)
                            if not any(word in content for word in ['징역', '집행']) and not re.search(r'(\d{1,3}(,\d{3})*)\s*원|(\d+)\s*만\s*원|(\d{1,3}(,\d{3})*)\s*환|(\d+)\s*만\s*환', content):
                                continue

                            # 징역 정보 추출
                            sentence_length = extract_sentence_length(content)
                            if sentence_length:
                                print(f"징역: {sentence_length}개월")
                                filtered_dataset_df2_cleaned.at[index, '징역(개월)'] = sentence_length

                            # 집행유예 정보 추출
                            months_before_jiphaeng = extract_months_before_jiphaeng(content)
                            if months_before_jiphaeng:
                                print(f"집행유예: {months_before_jiphaeng}개월")
                                filtered_dataset_df2_cleaned.at[index, '집행유예(개월)'] = months_before_jiphaeng

                            # 벌금 정보 추출
                            fine_amount = extract_fine_amount(content)
                            if fine_amount:
                                print(f"벌금: {fine_amount}원")
                                filtered_dataset_df2_cleaned.at[index, '벌금'] = fine_amount

                            # 최근 형량을 추출하였기에 Mission Complete
                            if sentence_length or months_before_jiphaeng or fine_amount:
                              found+=1
                              break
                    else:
                        print(f"Failed to retrieve iframe content: {iframe_response.status_code}")
                else:
                    print("No iframe found")
        except requests.exceptions.RequestException as e:
            print(f"Error for URL {url}: {e}")

# 결과 출력
filtered_dataset_df2_cleaned.to_csv('filtered_dataset_df2_cleaned2.csv', index=False)

Processing cases:   5%|▌         | 262/5008 [02:41<58:08,  1.36it/s]

징역: 12개월


Processing cases:   5%|▌         | 267/5008 [02:44<51:55,  1.52it/s]

징역: 24개월


Processing cases:   6%|▌         | 305/5008 [03:06<43:21,  1.81it/s]

...

벌금: 79041439원


Processing cases: 100%|█████████▉| 5007/5008 [1:04:59<00:00,  1.37it/s]

벌금: 10000000000원


Processing cases: 100%|██████████| 5008/5008 [1:05:00<00:00,  1.28it/s]

벌금: 12693699원


In [ ]:
found

1745

In [ ]:
filtered_dataset_df2_cleaned = filtered_dataset_df2_cleaned.dropna(subset=['징역(개월)', '집행유예(개월)', '벌금'], how='all')

In [ ]:
pd.reset_option('display.max_colwidth')
filtered_dataset_df2_cleaned

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,...,전문,분류,판결,징역(개월),집행유예(개월),벌금,원심결과,날짜,URL,원심번호
261,155040,폭력행위등처벌에관한법률위반등,71도399,19710524,선고,대법원,형사,판결,노동쟁의와 정당행위,경비원은 상사의 명령에 의하여 주로 경비업무등 노무를 제공하는 직분을 가지고 있는 ...,...,"【피 고 인】\n【상 고 인】\n검사\n【원심판결】\n제1심 부산지방, 제2심 대구...",폭행,"원판결중 피고인 김정중, 김동식이가 피고인 허재업과 공동하여 1969. 8. 19....",12.0,NaN,NaN,"제1심 부산지방, 제2심 대구고등 1971. 2. 4. 선고 70노244 판결",1971.02.04.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,70노244
266,155027,조세범처벌법위반,71도742,19710531,선고,대법원,형사,판결,과세대상과 납세의무자 확정의 잘못으로 과세처분이 당연 무효인 경우와 체납범의 성립.,과세대상과 납세의무자 확정이 잘못되어 당연무효한 과세에 대하여는 체납이 문제될 여지...,...,"【피 고 인】\n【상 고 인】\n검사\n【원심판결】\n제1심 전주지방, 제2심 전주...",조세범처벌법,상고를 기각한다.,24.0,NaN,NaN,"제1심 전주지방, 제2심 전주지방 1971. 3. 9. 선고 69노957 판결",1971.03.09.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,69노957
304,154722,"특수강도,상해",71도2114,19720131,선고,대법원,형사,판결,피해자의 뒤를 따라가다가 그 등을 발로 세게 차서 상해를 입힌 연휴 물건을 빼앗은 ...,피해자의 뒤를 따라가다가 그 등을 발로 세게 차서 상해를 입힌 연휴 물건을 빼앗은 ...,...,"【피고인, 상고인】\n【원심판결】\n제1심, 서울형사지방, 제2심 서울고등 1971...",강도,상고를 기각한다.\n상고후의 구금일수중 70일을 본형에 산입한다.,24.0,36.0,3.000000e+06,"제1심, 서울형사지방, 제2심 서울고등 1971. 10. 25. 선고 71노722 판결",1971.10.25.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,71노722
339,208011,퇴직금,72다2425,19730327,선고,대법원,민사,판결,교재연구비가 임금인지의 여부,교재연구비를 임금이라고 볼 것인가는 그 보수규정 근로협약 취업규칙 등에 의하여 명시...,...,"【원고, 피상고인】\n원고 소송대리인 변호사 김용진, 이상혁\n【피고, 상고인】\n...",근로기준법,원판결을 파기하고 사건을 서울고등법원에 환송한다.,NaN,NaN,9.880000e+05,서울고등법원 1972.11.28. 선고 72나2203 판결,1972.11.28.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,72나2203
351,92029,업무상횡령·업무상배임·위증,73도2339,19731127,선고,대법원,형사,판결,변조된 수표에 대하여 피고인이 그 수표의 발행자측으로서 이상없는 수표라고 말하며 그...,수표의 발행일자를 변조하면서 거기에 정정인이 찍혀있지 않았기 때문에 지급인인 농업협...,...,"【피고인, 상고인】\n【원심판결】\n서울형사지방법원 1973.7.27. 선고 73노...",횡령,이 상고를 모두 기각한다.,6.0,12.0,NaN,서울형사지방법원 1973.7.27. 선고 73노1255 판결,1973.07.27.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,73노1255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,234711,노동조합및노동관계조정법위반·근로기준법위반·남녀고용평등과일·가정양립지원에관한법률위반,2020도16431,20230427,선고,대법원,형사,판결,[1] 구 근로기준법 제51조 제1항에서 정한 탄력적 근로시간제는 취업규칙에 의하여...,[1] 구 근로기준법(2017. 11. 28. 법률 제15108호로 개정되기 전의 ...,...,【피 고 인】\n피고인\n【상 고 인】\n피고인 및 검사\n【변 호 인】\n변호사 ...,근로기준법,원심판결 중 유죄 부분(이유무죄 부분 포함)과 연장근로수당 미지급으로 인한 근로기준...,NaN,NaN,5.000000e+06,인천지법 2020. 11. 5. 선고 2019노3882 판결,2020.11.05.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2019노3882
5004,234703,마약류관리에관한법률위반(향정),2023도2102,20230427,선고,대법원,형사,판결,[1] 형사소송법 제254조 제4항에서 공소사실의 특정을 요구하는 취지 및 범죄 ‘...,"[1] 공소사실의 기재는 범죄의 일시, 장소와 방법을 명시하여 사실을 특정할 수 있...",...,【피 고 인】\n피고인\n【상 고 인】\n피고인\n【변 호 인】\n변호사 윤대진\n...,마약류관리에관한법률,"원심판결을 파기하고, 사건을 대구지방법원에 환송한다.",4,NaN,2.000000e+05,대구지법 2023. 1. 18. 선고 2022노3872 판결,2023.01.18.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2022노3872
5005,234647,근로에관한소송[구파견법에따라직접고용청구권을취득한파견근로자가사용사업주를상대로고용의사표...,2021다229601,20230427,선고,대법원,민사,판결,"[1] 사용사업주에 대한 회생절차개시결정이 있는 경우, 회생절차개시결정 전에 발생한...",[1] 구 파견근로자 보호 등에 관한 법률(2012. 2. 1. 법률 제11279호...,...,"【원고, 피상고인】\n원고 1 외 5인 (소송대리인 법무법인 중심 담당변호사 류재율...",근로기준법,"원심판결 중 원고 1, 원고 2, 원고 4, 원고 5, 원고 6에 대한 부분을 파기...",NaN,NaN,7.904144e+07,서울고법 2021. 4. 9. 선고 (춘천)2020나1108 판결,2021.04.09.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2020나1108
5006,234695,부당이득금[파생상품거래에대해투자매매업자의착오로인한취소가능여부가문제된사건],2017다227264,20230427,선고,대법원,민사,판결,[1] 민법 제109조 제1항 단서에서 정한 ‘중대한 과실’의 의미 / 의사표시의 ...,[1] 민법 제109조 제1항은 법률행위 내용의 중요 부분에 착오가 있는 때에는 그...,...,"【원고, 상고인】\n한맥투자증권 주식회사의 소송수계인 파산채무자 한맥투자증권 주식회...",사기,상고를 기각한다. 상고비용은 원고가 부담한다.,NaN,NaN,1.000000e+10,서울고법 2017. 4. 7. 선고 2015나2055371 판결,2017.04.07.,[https://www.law.go.kr/DRF/lawService.do?OC=sp...,2015나2055371


In [ ]:
import subprocess
csv_filename = 'filtered_dataset.csv'
tar_filename = 'filtered_dataset.tar.gz'
subprocess.run(['tar', '-czvf', tar_filename, csv_filename])

# SSH 키 파일 권한 설정
chmod_command = ['chmod', '400', './news-analysis-key-pair.pem']
subprocess.run(chmod_command)

# 결과 확인
ls_command = ['ls', '-l', './news-analysis-key-pair.pem']
result = subprocess.run(ls_command, capture_output=True, text=True)
print(result.stdout)

# 압축 파일 전송
scp_command = [
    'scp',
    '-o', 'StrictHostKeyChecking=no',
    '-i', './news-analysis-key-pair.pem',
    'filtered_dataset.tar.gz',
    'ubuntu@15.164.119.80:/home/ubuntu/'
]

result = subprocess.run(scp_command, capture_output=True, text=True)

ssh_key_path = './news-analysis-key-pair.pem'  # SSH 키 파일 경로
ec2_user = 'ubuntu'
ec2_host = '15.164.119.80'
remote_path = '/home/ubuntu/'

scp_command = [
    'scp',
    '-i', ssh_key_path,
    tar_filename,
    f'{ec2_user}@{ec2_host}:{remote_path}'
]

# 전송 시도 및 오류 출력
result = subprocess.run(scp_command, capture_output=True, text=True)
if result.returncode != 0:
    print("Error:", result.stderr)
else:
    print("File transferred successfully")

In [ ]:
filtered_dataset_df2_cleaned['URL']

SyntaxError: invalid decimal literal (<ipython-input-33-13d3a69cd044>, line 1)

In [ ]:
filtered_dataset_df2_cleaned['URL'].to_csv('123.txt', index=False)

In [ ]:
nan=filtered_dataset_df[filtered_dataset_df['징역(개월)'].isna()&filtered_dataset_df['집행유예(개월)'].isna()&filtered_dataset_df['벌금'].isna()]
len(nan)

16166

In [ ]:
# '판례정보일련번호'를 기준으로 값을 업데이트
for index, row in filtered_dataset_df2_cleaned.iterrows():
    matching_index = filtered_dataset_df[filtered_dataset_df['판례정보일련번호'] == row['판례정보일련번호']].index
    if not matching_index.empty:
        idx = matching_index[0]
        for column in ['징역(개월)', '집행유예(개월)', '벌금']:
            if pd.notna(row[column]):
                filtered_dataset_df.at[idx, column] = row[column]

In [ ]:
nan=filtered_dataset_df[filtered_dataset_df['징역(개월)'].isna()&filtered_dataset_df['집행유예(개월)'].isna()&filtered_dataset_df['벌금'].isna()]
len(nan)

14421

# 형량이 존재하는 데이터 개수

In [ ]:
len(filtered_dataset_df)-len(nan)

4988

In [ ]:
# '징역(개월)', '집행유예(개월)', '벌금' 열이 모두 NaN인 행을 삭제
filtered_dataset_df = filtered_dataset_df.dropna(subset=['징역(개월)', '집행유예(개월)', '벌금'], how='all')
filtered_dataset_df['사건개수'] = filtered_dataset_df['사건명'].apply(lambda x: x.count(',') + x.count('·') + x.count('ㆍ') + 1)

In [ ]:
filtered_rows = filtered_dataset_df[(filtered_dataset_df['사건명'].str.contains('위험운전', na=False))&(filtered_dataset_df['사건명'].str.contains('사고후미조치', na=False))]
filtered_rows

,판례정보일련번호,사건명,사건번호,선고일자,선고,법원명,사건종류명,판결유형,판시사항,판결요지,참조조문,참조판례,전문,분류,판결,징역(개월),집행유예(개월),벌금,사건개수
84068,221827,특정범죄가중처벌등에관한법률위반(운전자폭행등)·재물손괴·특정범죄가중처벌등에관한법률위반...,"2021고합123, 136(병합), 168(병합), 217(병합), 2021초기682",20210924,선고,부산지방법원,형사,판결,None,None,None,None,"【피 고 인】\n피고인\n【검 사】\n박보경, 윤동환, 하재무, 이자영(기소), 김...",도로교통법,피고인을 징역 4년에 처한다.\n배상신청인의 배상신청을 각하한다.,48,NaN,NaN,8
84428,221787,특정범죄가중처벌등에관한법률위반(운전자폭행등)(인정된죄명상해)·재물손괴·특정범죄가중처...,2021노393,20220113,선고,부산고등법원,형사,판결,None,None,None,None,"【피 고 인】\n피고인\n【항 소 인】\n피고인\n【검 사】\n박보경, 윤동환, 하...",도로교통법,원심판결 중 배상명령신청 각하 부분을 제외한 나머지를 파기한다.\n피고인을 징역 3...,36,NaN,NaN,7
84784,220849,특정범죄가중처벌등에관한법률위반(운전자폭행등)(인정된죄명:상해)·재물손괴·특정범죄가중...,2022도1013,20220428,선고,대법원,형사,판결,[1] 특정범죄 가중처벌 등에 관한 법률 제5조의10에서 정한 ‘자동차’가 도로교통...,[1] 특정범죄 가중처벌 등에 관한 법률(이하 ‘특정범죄가중법’이라 한다) 제5조의...,[1] 특정범죄 가중처벌 등에 관한 법률 제5조의10 / [2] 특정범죄 가중처벌 ...,[2] 대법원 2008. 12. 11. 선고 2008도4375 판결,【피 고 인】\n피고인\n【상 고 인】\n피고인 및 검사\n【변 호 인】\n변호사 ...,도로교통법,상고를 모두 기각한다.,36,NaN,NaN,7


In [ ]:
import pandas as pd
import tarfile

# 데이터프레임을 CSV 파일로 저장
csv_filename = 'filtered_dataset_df.csv'
filtered_dataset_df.to_csv(csv_filename, index=False)

# CSV 파일을 tar.gz 파일로 압축
tar_filename = 'filtered_dataset_df.tar.gz'
with tarfile.open(tar_filename, 'w:gz') as tar:
    tar.add(csv_filename, arcname=csv_filename)

print(f"{tar_filename} 파일이 생성되었습니다.")

filtered_dataset_df.tar.gz 파일이 생성되었습니다.
